In [ ]:
#!import "//ifrs17ce/dev/CalculationEngine"

In [ ]:
var gic = "DT1.1";

# Test Load Current and Previous Parameter for Yield Curve

In [ ]:
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());

In [ ]:
public async Task CheckLoadCurrentAndPreviousParameterForYieldCurveAsync(Args args, IEnumerable<YieldCurve> testData, 
                                                                         (int Year, int Month) expectedCurrentPeriod, 
                                                                         (int Year, int Month) expectedPreviousPeriod)
{
    await Workspace.UpdateAsync(testData);
    var eurCurrentAndPreviousYieldCurve = (await Workspace.LoadCurrentAndPreviousParameterAsync<YieldCurve>(args, x => x.Currency))["EUR"];
    
    //Check Current Period
    eurCurrentAndPreviousYieldCurve[CurrentPeriod].Year.Should().Be(expectedCurrentPeriod.Year);
    eurCurrentAndPreviousYieldCurve[CurrentPeriod].Month.Should().Be(expectedCurrentPeriod.Month);
    
    //Check Previous Period
    eurCurrentAndPreviousYieldCurve[PreviousPeriod].Year.Should().Be(expectedPreviousPeriod.Year);
    eurCurrentAndPreviousYieldCurve[PreviousPeriod].Month.Should().Be(expectedPreviousPeriod.Month);
    
    await Workspace.DeleteAsync(Workspace.Query<YieldCurve>().ToArray());
}

In [ ]:
var args = new Args("CH",2020,9,Periodicity.Monthly,null);
var testData = new YieldCurve{ Currency = "EUR", Year = 2020, Month = 9};
await CheckLoadCurrentAndPreviousParameterForYieldCurveAsync(args, testData.RepeatOnce(), (2020, 9),(2020, 9));

In [ ]:
var args = new Args("CH",2020,9,Periodicity.Monthly,null);
var testData = new YieldCurve{ Currency = "EUR", Year = 2019, Month = 12};
await CheckLoadCurrentAndPreviousParameterForYieldCurveAsync(args, testData.RepeatOnce(), (2019, 12), (2019, 12));

In [ ]:
var args = new Args("CH",2020,9,Periodicity.Monthly,null);
var testData = new YieldCurve[] {new YieldCurve{ Currency = "EUR", Year = 2020, Month = 6}, 
                                 new YieldCurve{ Currency = "EUR", Year = 2020, Month = 3}};
await CheckLoadCurrentAndPreviousParameterForYieldCurveAsync(args, testData, (2020, 6), (2020, 6));

In [ ]:
var args = new Args("CH",2020,9,Periodicity.Monthly,null);
var testData = new YieldCurve[] {new YieldCurve{ Currency = "EUR", Year = 2020, Month = 9}, 
                                 new YieldCurve{ Currency = "EUR", Year = 2020, Month = 6}};
await CheckLoadCurrentAndPreviousParameterForYieldCurveAsync(args, testData, (2020, 9), (2020, 9));

In [ ]:
var args = new Args("CH",2020,9,Periodicity.Monthly,null);
var testData = new YieldCurve[] {new YieldCurve{ Currency = "EUR", Year = 2019, Month = 9}, 
                                 new YieldCurve{ Currency = "EUR", Year = 2020, Month = 9}};
await CheckLoadCurrentAndPreviousParameterForYieldCurveAsync(args, testData, (2020, 9), (2019, 9));

In [ ]:
var args = new Args("CH",2020,9,Periodicity.Monthly,null);
var testData = new YieldCurve[] {new YieldCurve{ Currency = "EUR", Year = 2019, Month = 12}, 
                                 new YieldCurve{ Currency = "EUR", Year = 2019, Month = 9}};
await CheckLoadCurrentAndPreviousParameterForYieldCurveAsync(args, testData, (2019, 12), (2019, 12));

In [ ]:
var args = new Args("CH",2020,9,Periodicity.Monthly,null);
var testData = new YieldCurve[] {new YieldCurve{ Currency = "EUR", Year = 2019, Month = 12}, 
                                 new YieldCurve{ Currency = "EUR", Year = 2018, Month = 9}};
await CheckLoadCurrentAndPreviousParameterForYieldCurveAsync(args, testData, (2019, 12), (2019, 12));

In [ ]:
var args = new Args("CH",2020,9,Periodicity.Monthly,null);
var testData = new YieldCurve[] {new YieldCurve{ Currency = "EUR", Year = 2019, Month = 6}, 
                                 new YieldCurve{ Currency = "EUR", Year = 2016, Month = 9},
                                 new YieldCurve{ Currency = "EUR", Year = 2016, Month = 6}};
await CheckLoadCurrentAndPreviousParameterForYieldCurveAsync(args, testData, (2019, 6), (2019, 6));

In [ ]:
var args = new Args("CH",2020,9,Periodicity.Monthly,null);
var testData = new YieldCurve[] {new YieldCurve{ Currency = "EUR", Year = 2020, Month = 6}, 
                                 new YieldCurve{ Currency = "EUR", Year = 2020, Month = 3}, 
                                 new YieldCurve{ Currency = "EUR", Year = 2015, Month = 9},
                                 new YieldCurve{ Currency = "EUR", Year = 2016, Month = 3}};
await CheckLoadCurrentAndPreviousParameterForYieldCurveAsync(args, testData, (2020, 6), (2016, 3));

In [ ]:
var args = new Args("CH",2020,9,Periodicity.Monthly,null);
var testData = new YieldCurve[] {new YieldCurve{ Currency = "EUR", Year = 2020, Month = 6}, 
                                 new YieldCurve{ Currency = "EUR", Year = 2020, Month = 3}, 
                                 new YieldCurve{ Currency = "EUR", Year = 2021, Month = 9},
                                 new YieldCurve{ Currency = "EUR", Year = 2016, Month = 3}};

await CheckLoadCurrentAndPreviousParameterForYieldCurveAsync(args, testData, (2020, 6), (2016, 3));

# Test Data Node State Query

In [ ]:
public async Task CheckLoadDataNodeStateAsync(Args args, IEnumerable<DataNodeState> testData, bool isExpectedToBeActive)
{
    await Workspace.Partition.SetAsync<PartitionByReportingNode>(args);
    await Workspace.UpdateAsync(testData);
    var isActive = (await Workspace.LoadDataNodeStateAsync(args)).Keys.Contains(gic);
    
    isActive.Should().Be(isExpectedToBeActive);
    
    Workspace.Reset(x => x.ResetCurrentPartitions());
}

In [ ]:
var args = new Args("CH",2020,12,Periodicity.Monthly,null);
var testData = new DataNodeState[] {new DataNodeState{ DataNode = gic, Year = 2019, Month = 12, State = State.Active },
                                    new DataNodeState{ DataNode = gic, Year = 2020, Month = 6, State = State.Inactive }};
await CheckLoadDataNodeStateAsync(args, testData, false);

In [ ]:
var args = new Args("CH",2020,12,Periodicity.Monthly,null);
var testData = new DataNodeState[] {new DataNodeState{ DataNode = gic, Year = 2020, Month = 3, State = State.Active },
                                    new DataNodeState{ DataNode = gic, Year = 2020, Month = 9, State = State.Inactive }};
await CheckLoadDataNodeStateAsync(args, testData, false);

# Test Load Yield Curve

In [ ]:
public async Task CheckLoadYieldCurveAsync(Args args, IEnumerable<YieldCurve> testData, 
                                           (int year , int month) dataNodeYearAndMonth, 
                                           double expectedCurrentPreviousFirstYcValue,
                                           double expectedCurrentFirstYcValue,
                                           double expectedLockedFirstYcValue)
{
    await Workspace.Partition.SetAsync<PartitionByReportingNode>(args);
    await Workspace.UpdateAsync(testData);
    await Workspace.UpdateAsync<ReportingNode>(new ReportingNode[]{new ReportingNode{Currency="EUR",SystemName="CH"}});
    
    var dataNodes = new DataNodeData[]{new DataNodeData{ValuationApproach="BBA", Year = dataNodeYearAndMonth.year, Month = dataNodeYearAndMonth.month, DataNode = gic, ContractualCurrency = "EUR"}};
    var eurLockedYieldCurve = (await LoadLockedInYieldCurveAsync(Workspace, args, dataNodes));
    var eurCurrentYieldCurve = (await LoadCurrentYieldCurveAsync(Workspace, args, dataNodes));
    
    Workspace.Reset(x => x.ResetCurrentPartitions());
    
    var errors = new List<string>();
    var currentPrevPeriod = eurCurrentYieldCurve["EUR"][PreviousPeriod].Values.First();
    if(Math.Abs(currentPrevPeriod - expectedCurrentPreviousFirstYcValue) > 1E-8)
        errors.Add( $"Current YieldCurve for Previous Period not matching:\n Expected {expectedCurrentPreviousFirstYcValue} \n Computed {currentPrevPeriod}." );
    
    var currentCurrPeriod = eurCurrentYieldCurve["EUR"][CurrentPeriod].Values.First();
    if(Math.Abs(currentCurrPeriod - expectedCurrentFirstYcValue) > 1E-8)
        errors.Add( $"Current YieldCurve for Current Period not matching:\n Expected {expectedCurrentFirstYcValue} \n Computed {currentCurrPeriod}." );
    
    var lockedYc = eurLockedYieldCurve[gic].Values.First();
    if(Math.Abs(lockedYc - expectedLockedFirstYcValue) > 1E-8)
        errors.Add( $"LockedIn YieldCurve not matching:\n Expected {expectedLockedFirstYcValue} \n Computed {lockedYc}." );
     
    if(errors.Any()) ApplicationMessage.Log(Error.Generic, string.Join("\n", errors));        
}

In [ ]:
var args = new Args("CH",2020,9,Periodicity.Monthly,null);
var testData = new YieldCurve[] {new YieldCurve{ Currency = "EUR", Year = 2020, Month = 6, Values = new double[]{0.1,0.2,0.3,0.4,0.5,0.6} }, 
                                 new YieldCurve{ Currency = "EUR", Year = 2020, Month = 3, Values = new double[]{2.1,2.2,2.3,2.4,2.5,2.6} }, 
                                 new YieldCurve{ Currency = "EUR", Year = 2021, Month = 9, Values = new double[]{4.1,4.2,4.3,4.4,4.5,4.6} },
                                 new YieldCurve{ Currency = "EUR", Year = 2016, Month = 3, Values = new double[]{6.1,6.2,6.3,6.4,6.5,6.6} }};

await CheckLoadYieldCurveAsync(args, testData, (2016, 6), 6.1, 0.1, 6.1);

# Test Load Data Node Parameter

In [ ]:
var gic2 = "gic2";
var xgic = "xgic";
var gric1 = "gric1";

In [ ]:
public async Task CheckLoadInterDataNodeParameterAsync(Args args, IEnumerable<InterDataNodeParameter> testData, 
                                                       int previousYear,
                                                       int? currentYear = null)
{
    currentYear = currentYear ?? previousYear;
    await Workspace.Partition.SetAsync<PartitionByReportingNode>(args);
    await Workspace.UpdateAsync(testData);
        
    var interDataNodeParameters = await LoadInterDataNodeParametersAsync(Workspace, args);
   
    //Check Keys
    var uniqueLinks = testData.Select(x => x.DataNode).Concat(testData.Select(x => x.LinkedDataNode)).ToHashSet();
    uniqueLinks.Intersect(interDataNodeParameters.Keys).Count().Should().Be(uniqueLinks.Count());
    
    //CheckInnerDictionaries
    var currentYearData = testData.Where(x => x.Year == currentYear);
    var previousYearData = testData.Where(x => x.Year == previousYear);
    
    var errors = new List<string>();
    foreach(var dn in uniqueLinks)
    {
        //Set up BM counts
        var currentLinks = currentYearData.Select(x => x.DataNode).Concat(currentYearData.Select(x => x.LinkedDataNode));
        var currentLinksCount = currentLinks.Where(x => x == dn).Count();
        var prevlinks = previousYearData.Select(x => x.DataNode).Concat(previousYearData.Select(x => x.LinkedDataNode));
        var previousLinksCount = prevlinks.Where(x => x == dn).Count();
        
        //Current Period
        var computedCount = interDataNodeParameters[dn][CurrentPeriod].Count();

        if (currentYearData.Any())
        {
            if(computedCount != currentLinksCount)
                errors.Add( $"Current DataNode parameter count for {dn} not matching:\n Expected {currentLinksCount} \n Computed {computedCount}." );
        }
        else if(computedCount != previousLinksCount)
                errors.Add( $"Current DataNode parameter count for {dn} not matching:\n Expected {previousLinksCount} \n Computed {computedCount}." );
        
        foreach (var param in interDataNodeParameters[dn][CurrentPeriod])
        {
            var linkedDn = param.DataNode == dn ? param.LinkedDataNode : param.DataNode;
            var isLinkInPrevPeriod = previousYearData.Any(x => x.DataNode == dn && x.LinkedDataNode == linkedDn || x.DataNode == linkedDn && x.LinkedDataNode == dn);
            var isLinkInCurrPeriod = currentYearData.Any(x => x.DataNode == dn && x.LinkedDataNode == linkedDn || x.DataNode == linkedDn && x.LinkedDataNode == dn); 
            
           if(isLinkInPrevPeriod && !isLinkInCurrPeriod)
            {    
                if (param.ReinsuranceCoverage != ((double)previousYear)/1000)
                    errors.Add( $"Current Reinsurance Coverage for {dn} linked to {linkedDn} not matching:\n Expected {((double)previousYear)/1000} \n Computed {param.ReinsuranceCoverage}." );
            }
            else if( param.ReinsuranceCoverage != ((double)currentYear)/1000)
                errors.Add( $"Current Reinsurance Coverage for {dn} linked to {linkedDn} not matching:\n Expected {((double)currentYear)/1000} \n Computed {param.ReinsuranceCoverage}." );
        }

        //Previous Period
        computedCount = interDataNodeParameters[dn][PreviousPeriod].Count();
       
        if (currentYearData.Any())
        {
            if(computedCount != currentLinksCount)
                errors.Add( $"Previous DataNode parameter count for {dn} not matching:\n Expected {currentLinksCount} \n Computed {computedCount}." );
        }
        else if(computedCount != previousLinksCount)
                errors.Add( $"Previous DataNode parameter count for {dn} not matching:\n Expected {previousLinksCount} \n Computed {computedCount}." );
        
        foreach (var param in interDataNodeParameters[dn][PreviousPeriod])
        {
            var linkedDn = param.DataNode == dn ? param.LinkedDataNode : param.DataNode;
            var isLinkInPrevPeriod = previousYearData.Any(x => x.DataNode == dn && x.LinkedDataNode == linkedDn || x.DataNode == linkedDn && x.LinkedDataNode == dn);
            var isLinkInCurrPeriod = currentYearData.Any(x => x.DataNode == dn && x.LinkedDataNode == linkedDn || x.DataNode == linkedDn && x.LinkedDataNode == dn); 
            
           if(!isLinkInPrevPeriod && isLinkInCurrPeriod)
            {    
                if (param.ReinsuranceCoverage != ((double)currentYear)/1000)
                    errors.Add( $"Previous Reinsurance Coverage for {dn} linked to {linkedDn} not matching:\n Expected {((double)currentYear)/1000} \n Computed {param.ReinsuranceCoverage}." );
            }
            else if( param.ReinsuranceCoverage != ((double)previousYear)/1000)
                errors.Add( $"Previous Reinsurance Coverage for {dn} linked to {linkedDn} not matching:\n Expected {((double)previousYear)/1000} \n Computed {param.ReinsuranceCoverage}." );
        }
    }
    
    Workspace.Reset(x => x.ResetCurrentPartitions());
   
    if(errors.Any()) ApplicationMessage.Log(Error.Generic, string.Join("\n", errors));
}

In [ ]:
var args = new Args("CH",2020,9,Periodicity.Monthly,null);
int previousYear = 2019;
var testData = new InterDataNodeParameter[] 
                                {new InterDataNodeParameter{ DataNode = gic, LinkedDataNode = gric1, ReinsuranceCoverage = ((double)previousYear)/1000, Year = previousYear, Month = 3}, 
                                 new InterDataNodeParameter{ DataNode = gic2, LinkedDataNode = gric1, ReinsuranceCoverage = ((double)previousYear)/1000, Year = previousYear, Month = 3}, 
                                 new InterDataNodeParameter{ DataNode = gric1, LinkedDataNode = xgic, ReinsuranceCoverage = ((double)previousYear)/1000, Year = previousYear, Month = 3}, 
                                 };

await CheckLoadInterDataNodeParameterAsync(args, testData, previousYear);

In [ ]:
var args = new Args("CH",2020,9,Periodicity.Monthly,null);
int previousYear = 2019;
int currentYear = 2020;
var testData = new InterDataNodeParameter[] 
                                {
                                 new InterDataNodeParameter{ DataNode = gic, LinkedDataNode = gric1, ReinsuranceCoverage = ((double)previousYear)/1000, Year = previousYear, Month = 3}, 
                                 new InterDataNodeParameter{ DataNode = gric1, LinkedDataNode = xgic, ReinsuranceCoverage = ((double)previousYear)/1000, Year = previousYear, Month = 3}, 
                                 new InterDataNodeParameter{ DataNode = gic, LinkedDataNode = gric1, ReinsuranceCoverage = ((double)currentYear)/1000, Year = currentYear, Month = 9},
                                 new InterDataNodeParameter{ DataNode = gic2, LinkedDataNode = gric1, ReinsuranceCoverage = ((double)currentYear)/1000, Year = currentYear, Month = 9}, 
                                 new InterDataNodeParameter{ DataNode = gric1, LinkedDataNode = xgic, ReinsuranceCoverage = ((double)currentYear)/1000, Year = currentYear, Month = 9}, 
                                };

await CheckLoadInterDataNodeParameterAsync(args, testData, previousYear, currentYear);

In [ ]:
var args = new Args("CH",2020,9,Periodicity.Monthly,null);
int previousYear = 2019;
int currentYear = 2020;
var testData = new InterDataNodeParameter[] 
                                {
                                 new InterDataNodeParameter{ DataNode = gic, LinkedDataNode = gric1, ReinsuranceCoverage = ((double)previousYear)/1000, Year = previousYear, Month = 3}, 
                                 new InterDataNodeParameter{ DataNode = gric1, LinkedDataNode = xgic, ReinsuranceCoverage = ((double)previousYear)/1000, Year = previousYear, Month = 3}, 
                                };

await CheckLoadInterDataNodeParameterAsync(args, testData, previousYear, currentYear);

In [ ]:
var args = new Args("CH",2020,9,Periodicity.Monthly,null);
int previousYear = 2019;
int currentYear = 2020;
var testData = new InterDataNodeParameter[] 
                                {
                                 new InterDataNodeParameter{ DataNode = gic, LinkedDataNode = gric1, ReinsuranceCoverage = ((double)currentYear)/1000, Year = currentYear, Month = 3}, 
                                 new InterDataNodeParameter{ DataNode = gric1, LinkedDataNode = xgic, ReinsuranceCoverage = ((double)currentYear)/1000, Year = currentYear, Month = 3}, 
                                };

await CheckLoadInterDataNodeParameterAsync(args, testData, previousYear, currentYear);